In [13]:
from pyrqa.time_series import TimeSeries
from pyrqa.settings import Settings
from pyrqa.analysis_type import Classic, Cross
from pyrqa.neighbourhood import FixedRadius
from pyrqa.metric import EuclideanMetric
from pyrqa.computation import RQAComputation
from pyrqa.image_generator import ImageGenerator

import pandas as pd
import glob
import re

In [4]:
# This is example code
data_points = [0.1, 0.5, 1.3, 0.7, 0.8, 1.4, 1.6, 1.2, 0.4, 1.1, 0.8, 0.2, 1.3]
time_series = TimeSeries(data_points,
                         embedding_dimension=2,
                         time_delay=2)
settings = Settings(time_series,
                    analysis_type=Classic,
                    neighbourhood=FixedRadius(0.65),
                    similarity_measure=EuclideanMetric,
                    theiler_corrector=1)
computation = RQAComputation.create(settings,
                                    verbose=True)
result = computation.run()
result.min_diagonal_line_length = 2
result.min_vertical_line_length = 2
result.min_white_vertical_line_length = 2
print(result)

[Platform 'Apple']
Vendor: Apple
Version: OpenCL 1.2 (Feb 12 2022 01:56:48)
Profile: FULL_PROFILE
Extensions: cl_APPLE_SetMemObjectDestructor cl_APPLE_ContextLoggingFunctions cl_APPLE_clut cl_APPLE_query_kernel_names cl_APPLE_gl_sharing cl_khr_gl_event


[Device 'Apple M1']
Vendor: Apple
Type: 4
Version: OpenCL 1.2 
Profile: FULL_PROFILE
Max Clock Frequency: 1000
Global Mem Size: 11453251584
Address Bits: 64
Max Compute Units: 8
Max Work Group Size: 256
Max Work Item Dimensions: 3
Max Work Item Sizes: [256, 256, 256]
Local Mem Size: 32768
Max Mem Alloc Size: 2147483648
Extensions: cl_APPLE_SetMemObjectDestructor cl_APPLE_ContextLoggingFunctions cl_APPLE_clut cl_APPLE_query_kernel_names cl_APPLE_gl_sharing cl_khr_gl_event cl_khr_byte_addressable_store cl_khr_global_int32_base_atomics cl_khr_global_int32_extended_atomics cl_khr_local_int32_base_atomics cl_khr_local_int32_extended_atomics cl_khr_3d_image_writes cl_khr_image2d_from_buffer cl_khr_depth_images 


RQA Result:

Minimum diagona

In [10]:
log_path = 'data/hand_eye_combined_logs'
out_plots_path = 'out/plots'

In [11]:
dfs = []

for f in glob.glob(log_path + "/*_Normalized.txt"):
  d = pd.read_csv(f, sep="\s+")
  id = re.findall(".*Subject_(\d+)_.*", f)[0]
  d["id"] = id
  dfs.append(d)

df = pd.concat(dfs, ignore_index=True)

In [32]:
def cross_rqa(x, y, id,plot=True,verbose=True):
  data_points_x = x # [0.9, 0.1, 0.2, 0.3, 0.5, 1.7, 0.4, 0.8, 1.5]
  time_series_x = TimeSeries(data_points_x,
                            embedding_dimension=2,
                            time_delay=1)
  data_points_y = y # [0.3, 1.3, 0.6, 0.2, 1.1, 1.9, 1.3, 0.4, 0.7, 0.9, 1.6]
  time_series_y = TimeSeries(data_points_y,
                            embedding_dimension=2,
                            time_delay=2)
  time_series = (time_series_x,
                time_series_y)
  settings = Settings(time_series,
                      analysis_type=Cross,
                      neighbourhood=FixedRadius(0.73),
                      similarity_measure=EuclideanMetric,
                      theiler_corrector=0)
  computation = RQAComputation.create(settings)
  result = computation.run()
  result.min_diagonal_line_length = 2
  result.min_vertical_line_length = 2
  result.min_white_vertical_line_length = 2

  if(plot):
    ImageGenerator.save_recurrence_plot(result.recurrence_matrix_reverse,
                                        f'%s/%s_cross_recurrence_plot.png' % (out_plots_path, id))

  if(verbose): print(result)

  return result

In [ ]:
for group_id, group_df in df.groupby(['id']):
    x = group_df['LeftY']
    y = group_df['RightY']
    result = cross_rqa(x=x,y=y,id=group_id,plot=False,verbose=False)
    group_df['LeftY_RightY_recurrence_rate'] = result.recurrence_rate
    group_df['LeftY_RightY_determinism'] = result.determinism
    group_df['LeftY_RightY_average_diagonal_line'] = result.average_diagonal_line
    group_df['LeftY_RightY_longest_diagonal_line'] = result.longest_diagonal_line
    group_df['LeftY_RightY_divergence'] = result.divergence
    group_df['LeftY_RightY_entropy_diagonal_lines'] = result.entropy_diagonal_lines
    group_df['LeftY_RightY_laminarity'] = result.laminarity
    group_df['LeftY_RightY_trapping_time'] = result.trapping_time
    group_df['LeftY_RightY_longest_vertical_line'] = result.longest_vertical_line
    group_df['LeftY_RightY_entropy_vertical_lines'] = result.entropy_vertical_lines
    group_df['LeftY_RightY_average_white_vertical_line'] = result.average_white_vertical_line
    group_df['LeftY_RightY_longest_white_vertical_line'] = result.longest_white_vertical_line
    group_df['LeftY_RightY_entropy_white_vertical_lines'] = result.entropy_white_vertical_lines
    group_df['LeftY_RightY_ratio_determinism_recurrence_rate'] = result.ratio_determinism_recurrence_rate
    group_df['LeftY_RightY_ratio_laminarity_determinism'] = result.ratio_laminarity_determinism

In [63]:
df.groupby(['id']).filter(lambda g: g.id[0] == '03')

KeyError: 0